In [1]:
import pyodbc
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import paramiko
import os
import time
from datetime import datetime, timedelta
import numpy as np


In [ ]:
import os
import time
import warnings
import pyodbc
import pandas as pd
import numpy as np
from tqdm import tqdm
import paramiko
from datetime import datetime, timedelta
from IPython.display import clear_output
warnings.filterwarnings('ignore')

# Constants
SERVER_NAME=""
DATABASE=""
UID=""
PWD=""
BATCH_SIZE=2000
FILEMAN_IP=""
FILEMAN_USER=""
FILEMAN_PASSWORD=""
VISTA_USERNAME=""
VISTA_PASSWORD=""
FILEMAN_SETTING_FILE_PATH="fileman_labs_conditions2.xlsx"

def establish_connection():
    """Establish a connection to the SQL Server."""
    try:
        conn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};'
                              f'SERVER={SERVER_NAME};'
                              f'DATABASE={DATABASE}; UID={UID}; PWD={PWD};')
        print("Connection successful!")
        return conn
    except pyodbc.Error as e:
        print("Connection error:", e)
        return None

def get_max_value_from_db(conn):
    """Retrieve the maximum value from the NUMBER column in the LAB table."""
    sql_get = "SELECT MAX([DATE REPORT COMPLETED]) FROM LAB;"
    cursor = conn.cursor()
    cursor.execute(sql_get)
    max_value = cursor.fetchone()[0]
    cursor.close()
    return max_value.strftime("%m/%d/%Y@%H:%M") if max_value is not None else '01/01/1900'

def update_excel_with_max_value(file_path, max_value):
    """Update the Excel file with the maximum value retrieved from the database."""
    df = pd.read_excel(file_path, header=None)
    df.iloc[4, 0] = max_value
    return df

def generate_fileman_string(df):
    """Generate a string for the FileMan search based on the updated Excel file."""
    return ''.join(str(row[0]) + '\n' if not pd.isna(row[0]) else '\x0d' for _, row in df.iterrows())

def setup_ssh_connection(host, username, password, port=22):
    """Set up an SSH connection."""
    paramiko.util.log_to_file("patient_labs.log")
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, username=username, password=password, port=port)
    return ssh

def truncate_string(value, max_length):
    """Truncate string to the specified maximum length."""
    if isinstance(value, str) and len(value) > max_length:
        return value[:max_length]
    return value
string_columns_max_length = {
        "SPECIMEN TYPE": 100,
        "REQUESTING LOCATION": 100,
        "REQUESTING LOC/DIV": 100,
        "TEST_NAME": 50,
        "TEST_RESULT": 200
    }
            

def extract_data():
    """Main function to extract data by connecting to the database and performing SSH operations."""
    conn = establish_connection()
    if not conn:
        return

    max_value = get_max_value_from_db(conn)
    conn.close()
    global start_date
    global end_date
    print(f"The maximum Date in the column DATE REPORT COMPLETED is: {max_value}")
    start_date=pd.to_datetime(max_value,format="%m/%d/%Y@%H:%M")
    print(f"The start Date  is: {start_date}")
    end_date=pd.to_datetime(datetime.now()-timedelta(minutes=2))
    print(f"The end Date  is: {end_date}")
    
    fileman_search = update_excel_with_max_value(FILEMAN_SETTING_FILE_PATH, max_value)
    fileman = generate_fileman_string(fileman_search)
    
    steps = [
        '\x0d',
        VISTA_USERNAME,
        '\x0d',
        VISTA_PASSWORD,
        '\x0d',
        '\x0d',
        'Search File Entries\n',
        fileman
    ]

    ssh = setup_ssh_connection(FILEMAN_IP, FILEMAN_USER, FILEMAN_PASSWORD)
    channel = ssh.invoke_shell()

    if os.path.exists('output_labs.txt'):
        os.remove('output_labs.txt')
    
    
    for step in steps[:-1]:
        channel.send(step)
        while not channel.recv_ready():
            time.sleep(3)
        out = channel.recv(9999)
        print(out.decode('cp1256'))
        if 'invalid signon attempts.' in out.decode('cp1256') or 'Device/IP address is locked in' in out.decode('cp1256') or 'Do you really want to halt? YES//' in out.decode('cp1256') or 'Not a valid ACCESS CODE' in out.decode('cp1256') :
            print(f"Sorry, invalid signon attempts, i will retry after 1 min")
            time.sleep(60)
            main_function() 
    channel.send(steps[-1])
    prev_output = ""
    while True:
        out = channel.recv(9999).decode('cp1256')
        with open('output_labs.txt', 'a') as f:
            f.write(out)
        
        combined_output = prev_output + out
        prev_output = out

        if "MATCHES FOUND" in combined_output or "MATCH FOUND" in combined_output:
            ssh.close()
            break

    print("Data extraction ended.")
    
#     ssh.close()

def close_connection(conn):
    """Close the database connection."""
    if conn:
        conn.close()
        print("Connection closed.")

def parse_file(filename,conn):
    """Parse the file and insert data into the database."""
    batch_size =100000
    total_number = 0
    data_list = []
    count = 0
    num_lines = sum(1 for line in open(filename))
    print(f"Number of lines is : {num_lines}")
    pbar = tqdm(total=num_lines)
    
    columns = [
        "MRN", "DATE/TIME SPECIMEN TAKEN", "DATE REPORT COMPLETED","SPECIMEN TYPE", "REQUESTING LOCATION",
        "REQUESTING LOC/DIV", "TEST_NAME","TEST_RESULT"
    ]
    # List of specific labels to exclude
    exclude_labels = ["PARENT FILE", "LRDFN", "NAME", "REPORT ROUTING (LOCATION)", "REPORT ROUTING (PROVIDER)","VERIFY PERSON",
                 "ACCESSION","METHOD OR SITE","REQUESTING PERSON","ACCESSIONING INSTITUTION","UID","COMMENT","NEW PERSON CONVERSION",
                  "ORDERED TEST","RESP.INTERPRETATION","DATE/TIME OF DEATH (c)","NUMBER","Approved By:","SITE/SPECIMEN","WARD","LOCATION TYPE",
                   "SAMPLE TYPE",   
                 
                 
                 ]
    check=True
    with open(filename) as file:

        parsed_data = {}
        for line in file:
            pbar.update(1)
            split_line = line.split(" " * 3)
            for part in split_line: 
                if part.find(": MRN IS <<<<>>>>")!=-1:
                   

                    if data_list:
                        insert_data_batch(conn, data_list, columns,batch_size,start_date,end_date)
                        parsed_data = {}
                        data_list.clear()
                    parsed_data["MRN"] = part[17:-1]
                    

                else:
                    if ": " in part:
                        parts = part.split(": ")
                        key = parts[0].strip()
                        value = parts[1].strip()


                        if key=="DATE/TIME SPECIMEN TAKEN":
                            check=True
                            parsed_data["DATE/TIME SPECIMEN TAKEN"] = value
                            
                 
                        elif key=="PATHOLOGIST" or key=="PATHOLOGIST/CYTOTECHNOLOGIST" or key=="SPECIMEN SUBMITTED BY" or key=="MICROBIOLOGY ACCESSION":
                            check=False
                        else:
                            if check:
                                if key in columns:
                                    parsed_data[key] = value
                                elif key in exclude_labels:
                                    continue
                                else:
                                    parsed_data["TEST_NAME"] = key
                                    parsed_data["TEST_RESULT"] = value
                                    data_list.append(parsed_data.copy())
                                   
                                
    if data_list:
            insert_data_batch(conn, data_list, columns,batch_size,start_date,end_date)
            parsed_data = {}
            data_list.clear()     
            
            

def insert_data_batch(conn, data_list, columns,batch_size,start_date,end_date):
#     start_date=pd.to_datetime("01/01/1900")
#     end_date=pd.to_datetime("06/10/2024")
    """Insert data into the database in batches."""
    df= pd.DataFrame(data_list,columns=columns)
    # Replace '@' with ' '
    print(data_list)
    df['DATE/TIME SPECIMEN TAKEN'] = df['DATE/TIME SPECIMEN TAKEN'].str.replace('@', ' ')
    
    # Replace '24:00' with '00:00' and add one day
    mask_24 = df['DATE/TIME SPECIMEN TAKEN'].fillna('').str.contains(' 24:00')
    df.loc[mask_24, 'DATE/TIME SPECIMEN TAKEN'] = df.loc[mask_24, 'DATE/TIME SPECIMEN TAKEN'].str.replace(' 24:00', ' 00:00')
    df.loc[mask_24, 'DATE/TIME SPECIMEN TAKEN'] = pd.to_datetime(df.loc[mask_24, 'DATE/TIME SPECIMEN TAKEN'], format='%b %d, %Y %H:%M') + pd.DateOffset(days=1)

    # Convert to datetime, handling different formats
    df['DATE'] = pd.to_datetime(df['DATE/TIME SPECIMEN TAKEN'], format='%b %d, %Y %H:%M:%S', errors='coerce')
    df['DATE'] = pd.to_datetime(df['DATE/TIME SPECIMEN TAKEN'], format='%b %d, %Y %H:%M', errors='coerce').fillna(df['DATE'])

    # Update the original column with the converted dates
    df['DATE/TIME SPECIMEN TAKEN'] = df['DATE']

    # Drop the temporary 'DATE' column
    df.drop(columns=['DATE'], inplace=True)

    # Fill any remaining NaT values with a default value (e.g., NaN)
    df['DATE/TIME SPECIMEN TAKEN'].fillna(pd.to_datetime(np.nan), inplace=True)
    

    
    
    
        # Replace '@' with ' '
    df['DATE REPORT COMPLETED'] = df['DATE REPORT COMPLETED'].str.replace('@', ' ')
    
    # Replace '24:00' with '00:00' and add one day
    mask_24 = df['DATE REPORT COMPLETED'].fillna('').str.contains(' 24:00')
    df.loc[mask_24, 'DATE REPORT COMPLETED'] = df.loc[mask_24, 'DATE REPORT COMPLETED'].str.replace(' 24:00', ' 00:00')
    df.loc[mask_24, 'DATE REPORT COMPLETED'] = pd.to_datetime(df.loc[mask_24, 'DATE REPORT COMPLETED'], format='%b %d, %Y %H:%M') + pd.DateOffset(days=1)


    # Convert to datetime, handling different formats
    df['DATE'] = pd.to_datetime(df['DATE REPORT COMPLETED'], format='%b %d, %Y %H:%M:%S', errors='coerce')
    df['DATE'] = pd.to_datetime(df['DATE REPORT COMPLETED'], format='%b %d, %Y %H:%M', errors='coerce').fillna(df['DATE'])

    # Update the original column with the converted dates
    df['DATE REPORT COMPLETED'] = df['DATE']

    # Drop the temporary 'DATE' column
    df.drop(columns=['DATE'], inplace=True)


    

    df['MRN'] = pd.to_numeric(df['MRN'],errors='coerce')
    df['MRN'].fillna(0, inplace=True)
    df['MRN'] = df['MRN'].astype(int)
    
    df['TEST_RESULT'] = df['TEST_RESULT'].apply(lambda x: x[:100])
    df = df.where(pd.notnull(df), None)
    df = df.replace({np.nan: None})
    df=df.loc[(df['DATE REPORT COMPLETED']>start_date)&(df['DATE REPORT COMPLETED']<end_date)]
    for col, max_length in string_columns_max_length.items():
         df[col] = df[col].apply(lambda x: truncate_string(x, max_length))
    if not df.empty:        
        
        data_list = [tuple(row) for row in df.to_numpy()]
        cursor = conn.cursor()
        cursor.fast_executemany = True
        total_records = len(data_list)
        num_batches = (total_records // batch_size) + 1
        sql_insert = "INSERT INTO LAB (MRN, [DATE/TIME SPECIMEN TAKEN], [DATE REPORT COMPLETED], [SPECIMEN TYPE], [REQUESTING LOCATION], [REQUESTING LOC/DIV], [TEST_NAME], [TEST_RESULT]) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, total_records)
            batch_data = data_list[start_idx:end_idx]

            cursor.executemany(sql_insert, batch_data)
            conn.commit()
    

    
    
def main_function():
        try:
            extract_data()
            conn = establish_connection()
        except Exception as e:  
            print(f"Error encountered: {e}. Retrying in 1 minutes...")
            time.sleep(60)
            main_function()
#             if conn:
        try:
            'start parse file'
            parse_file('output_labs.txt',conn)

        except Exception as e:  
            print(f"Error encountered: {e}. Retrying in 1 minutes...")
            time.sleep(60)
            main_function()
#         finally:
#             close_connection(conn)
        print("The End")
        print(f"Success! The next loop will start in 1 minutes...")
        time.sleep(60)
        clear_output(wait=True)
if __name__ == "__main__":
    while True:
        try:
            main_function()
        except Exception as e:
            print(f"Error encountered: {e}. Retrying in 1 minutes...")
            main_function()
